In [1]:
train_r = r'C:\Users\hp\Downloads\Final_Assignment_Face Mask Dataset\Train'
test_r = r'C:\Users\hp\Downloads\Final_Assignment_Face Mask Dataset\Test'
valid_r = r'C:\Users\hp\Downloads\Final_Assignment_Face Mask Dataset\Validation'

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_gen = ImageDataGenerator( 
                              rescale = 1./255,
                              rotation_range = 40,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              zoom_range = 0.2,
                              horizontal_flip = True
                             )

In [4]:
test_gen = ImageDataGenerator(rescale=1./255)

In [5]:
train_data = train_gen.flow_from_directory(
 
    train_r,
    target_size=(128,128),
    batch_size=20,
    class_mode = 'binary'
)

Found 10000 images belonging to 2 classes.


In [6]:
test_data = test_gen.flow_from_directory(
 
    test_r,
    target_size=(128,128),
    batch_size=16,
    class_mode='binary'
)

Found 992 images belonging to 2 classes.


In [7]:
valid_data = test_gen.flow_from_directory(
 
    valid_r,
    target_size=(128,128),
    batch_size=16,
    class_mode='binary'
)

Found 800 images belonging to 2 classes.


In [8]:
from tensorflow.keras.applications import VGG19

In [9]:
con_base  = VGG19(weights = 'imagenet' , include_top = False ,input_shape =(128,128,3)  )

In [10]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [11]:
model = models.Sequential()
model.add(con_base)

In [12]:
model.add(layers.Flatten())
model.add(layers.Dense(512 , activation = 'relu'))
model.add(layers.Dense(1 , activation = 'sigmoid'))

In [13]:
from tensorflow.keras import optimizers
model.compile(optimizer = optimizers.RMSprop(lr=1e-4) , loss = 'binary_crossentropy', metrics = ['acc'] )

C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               4194816   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 24,219,713
Trainable params: 24,219,713
Non-trainable params: 0
_________________________________________________________________


In [15]:
con_base.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [16]:
from tensorflow.keras import callbacks

In [17]:
check_cb = callbacks.ModelCheckpoint("cnn_face-{epoch:02d}.h5")

In [ ]:
model_hist = model.fit_generator(
train_data,
    steps_per_epoch = 500,
    epochs=5,
    validation_data = valid_data,
    validation_steps = 50,
    callbacks = [check_cb]
)

C:\Users\hp\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
500/500 [==============================] - 6485s 13s/step - loss: 0.2316 - acc: 0.9073 - val_loss: 0.0297 - val_acc: 0.9950
Epoch 2/5
500/500 [==============================] - 5168s 10s/step - loss: 0.1078 - acc: 0.9735 - val_loss: 0.0177 - val_acc: 0.9937
Epoch 3/5
500/500 [==============================] - ETA: 0s - loss: 0.0975 - acc: 0.9760 

In [ ]:
import pandas as pd
pd.DataFrame(model_hist.history).plot(figsize = (10,10))
plt.grid()
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
model.evaluate(test_data, steps = 62)

In [ ]:
model.save("mask.h5p      ")